<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/network_connect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cdlib

In [10]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [11]:
#%reset -f

# 1.0 Call libraries
import pandas as pd
import numpy as np
import networkx as nx

# 1.01
import joblib
from collections import defaultdict

# 1.02
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 1.08 Misc
import os,gc , time,  glob
from os.path import exists
import feather

In [43]:
# Paths:
modulePath = "/gdrive/MyDrive/Colab_data_files/talkingData/03112022/"
dataPath = "/gdrive/MyDrive/Colab_data_files/skipgram/"
modelsPath = "/gdrive/MyDrive/Colab_data_files/skipgram/modelspath/"
pathToStoreProgress = "/gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/"

In [13]:
# 1.09 Home made modules
os.chdir(modulePath)
import network_features
from network_features import *
import utils
from utils import *

import importlib; importlib.reload(network_features)
import importlib; importlib.reload(utils)

<module 'utils' from '/gdrive/MyDrive/Colab_data_files/talkingData/03112022/utils.py'>

In [31]:
# 8.0 Read back pkl file and dictionary:
os.chdir(dataPath)
train = pd.read_pickle("seq.pkl")
train.head()

,a,b
0,455,213
1,61,265
2,180,181
3,426,357
4,455,453


In [30]:
# Read dictionary
filehandler = open("word_index.txt", 'r')
word2index = filehandler.read()
word2index

"{'football': 1, 'ball': 2, 'symptoms': 3, 'religion': 4, 'codes': 5, 'include': 6, 'various': 7, 'covid': 8, 'may': 9, 'people': 10, 'develop': 11, 'players': 12, 'rules': 13, 'virus': 14, 'religious': 15, 'team': 16, 'sports': 17, 'kicking': 18, 'goal': 19, 'word': 20, 'rugby': 21, 'common': 22, 'games': 23, 'disease': 24, 'religions': 25, 'popular': 26, 'known': 27, 'league': 28, 'gaelic': 29, 'many': 30, 'public': 31, 'th': 32, 'century': 33, 'spread': 34, 'elements': 35, 'two': 36, 'also': 37, 'either': 38, 'foot': 39, 'severe': 40, 'risk': 41, 'contaminated': 42, 'transcription': 43, 'social': 44, 'beliefs': 45, 'supernatural': 46, 'sacred': 47, 'life': 48, 'varying': 49, 'form': 50, 'association': 51, 'american': 52, 'canadian': 53, 'australian': 54, 'union': 55, 'share': 56, 'origins': 57, 'traditional': 58, 'played': 59, 'different': 60, 'world': 61, 'cultural': 62, 'influence': 63, 'british': 64, 'empire': 65, 'end': 66, 'distinct': 67, 'developing': 68, 'first': 69, 'several

In [32]:
org_columns = train.columns
cat_cols = ['a', 'b']

In [33]:
# 6.1 Create new columns (combinations of columns) in train
train = utils.create_cat_pairwise_cols(train , cat_cols)

In [34]:
train.head()

,a,b,a_p_b
0,455,213,455_213
1,61,265,61_265
2,180,181,180_181
3,426,357,426_357
4,455,453,455_453


In [35]:
# 6.11 The train columns are:
current_train_columns = train.columns
# 6.12 New columns created. These columns must be latter dropped
new_cols = list(set(current_train_columns).difference(set(org_columns)))
# 6.13 Revised cat cols (ie excluding hour etc)
cat_cols.extend(new_cols)
len(cat_cols)   # 15
cat_cols

['a', 'b', 'a_p_b']

In [36]:
# 8.1 Get all possibe permutation of our columns
#     Also decide which all centrality measures to cal
deg = 1    # Calculate degree centrality
eig = 1    # Calculate eigenvector centrality
pr =  1    # Calculate pagerank centrality
clu = 0    # Clustering coeff centrality
nf = None # nx.katz_centrality_numpy
cMeasures = [deg,eig,pr,clu, nf]

In [37]:
single_permute_list = utils.permute_colList(cat_cols,cMeasures)
single_permute_list

[('a', 'b', [1, 1, 1, 0, None]),
 ('a', 'a_p_b', [1, 1, 1, 0, None]),
 ('b', 'a', [1, 1, 1, 0, None]),
 ('b', 'a_p_b', [1, 1, 1, 0, None]),
 ('a_p_b', 'a', [1, 1, 1, 0, None]),
 ('a_p_b', 'b', [1, 1, 1, 0, None])]

In [38]:
len(single_permute_list)   # 210

6

In [39]:
spl = pd.DataFrame(single_permute_list, columns = ["colToProject", "intermediaryCol", "cMeasures?"])
spl.shape   # (210, 4)
spl

,colToProject,intermediaryCol,cMeasures?
0,a,b,"[1, 1, 1, 0, None]"
1,a,a_p_b,"[1, 1, 1, 0, None]"
2,b,a,"[1, 1, 1, 0, None]"
3,b,a_p_b,"[1, 1, 1, 0, None]"
4,a_p_b,a,"[1, 1, 1, 0, None]"
5,a_p_b,b,"[1, 1, 1, 0, None]"


In [40]:
record_sets = list(spl.to_records(index = False))

In [41]:
record_sets

[('a', 'b', list([1, 1, 1, 0, None])),
 ('a', 'a_p_b', list([1, 1, 1, 0, None])),
 ('b', 'a', list([1, 1, 1, 0, None])),
 ('b', 'a_p_b', list([1, 1, 1, 0, None])),
 ('a_p_b', 'a', list([1, 1, 1, 0, None])),
 ('a_p_b', 'b', list([1, 1, 1, 0, None]))]

In [42]:
fulllist = [record_sets]

In [45]:
X = train
m = 1
for k,j in enumerate(fulllist):
  print("Record set no is {} of {}th fulllist ".format(k+1,m+1))
  print("Set of records is: ", j)
  # Each outer loop creates a new sets of models
  fulllist_repeat_no = m + 1
  transYesNo = False  # True implies transformation
  X1_train, X1_test = network_features.get_store_features(X,j, transYesNo, None, fulllist_repeat_no , modelsPath, pathToStoreProgress)
  print("\nList done. X1_train shape ", X1_train.shape)
  del X1_train
  gc.collect()

Record set no is 1 of 2th fulllist 
Set of records is:  [('a', 'b', list([1, 1, 1, 0, None])), ('a', 'a_p_b', list([1, 1, 1, 0, None])), ('b', 'a', list([1, 1, 1, 0, None])), ('b', 'a_p_b', list([1, 1, 1, 0, None])), ('a_p_b', 'a', list([1, 1, 1, 0, None])), ('a_p_b', 'b', list([1, 1, 1, 0, None]))]
('a', 'b', list([1, 1, 1, 0, None]))

Next:  1 a b
No of unique values in  a  are:  480
Bipartite Network created. Time taken:  0.0007027904192606608 minutes
Bipartite Network projected. Time taken:  0.0034075021743774415 minutes
Model extracted. Time taken:  0.009365785121917724 minutes
Model stored to disk
train columns:  3
No test data
Done with a b
Time taken:  0.014222860336303711
Entry made in file progress.csv
('a', 'a_p_b', list([1, 1, 1, 0, None]))

Next:  2 a a_p_b
No of unique values in  a  are:  480
Bipartite Network created. Time taken:  0.0008696317672729492 minutes
Bipartite Network projected. Time taken:  0.00033295551935831705 minutes
Model extracted. Time taken:  0.0016080

In [80]:
def readStoredModels(pathToReadFrom, fileList):
  model_dfs = []
  os.chdir(pathToReadFrom)
  for file in fileList:
    df = pd.read_pickle(file)
    model_dfs.append(df)
  return model_dfs


In [81]:
os.chdir(modelsPath)
files = os.listdir()
file_list = [file for file in files if ("deg_a_ck" in file) or ("eig_a_ck" in file) or ("pr_a_ck" in file) ]


In [83]:
models = readStoredModels(modelsPath, file_list)

In [84]:
# Merge models
for i, model in enumerate(models):
  if i == 0:
    df = model
  else:
    df = pd.merge(df, model, left_index=True, right_index=True)  





In [85]:
df

,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,deg_a_ck_a_p_b,eig_a_ck_a_p_b,pr_a_ck_a_p_b
a,,,,,,
1.0,0.943633,0.136821,0.007628,0.0,0.043262,0.002083
2.0,0.787056,0.120828,0.006361,0.0,0.077953,0.002083
3.0,0.787056,0.117384,0.006390,0.0,-0.052496,0.002083
4.0,0.736952,0.111466,0.005999,0.0,-0.075497,0.002083
5.0,0.716075,0.112652,0.005796,0.0,-0.021041,0.002083
...,...,...,...,...,...,...
476.0,0.212944,0.038622,0.001909,0.0,0.054573,0.002083
477.0,0.187891,0.032655,0.001733,0.0,-0.071345,0.002083
478.0,0.219207,0.035809,0.001989,0.0,0.067491,0.002083


In [86]:
# Save dataframe:
os.chdir(dataPath)
df.to_pickle("word2vec.pkl")


In [88]:
# Load word2vec
os.chdir(dataPath)
pd.read_pickle("word2vec.pkl")

,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,deg_a_ck_a_p_b,eig_a_ck_a_p_b,pr_a_ck_a_p_b
a,,,,,,
1.0,0.943633,0.136821,0.007628,0.0,0.043262,0.002083
2.0,0.787056,0.120828,0.006361,0.0,0.077953,0.002083
3.0,0.787056,0.117384,0.006390,0.0,-0.052496,0.002083
4.0,0.736952,0.111466,0.005999,0.0,-0.075497,0.002083
5.0,0.716075,0.112652,0.005796,0.0,-0.021041,0.002083
...,...,...,...,...,...,...
476.0,0.212944,0.038622,0.001909,0.0,0.054573,0.002083
477.0,0.187891,0.032655,0.001733,0.0,-0.071345,0.002083
478.0,0.219207,0.035809,0.001989,0.0,0.067491,0.002083


In [89]:
# Read dicionary
word2index

"{'football': 1, 'ball': 2, 'symptoms': 3, 'religion': 4, 'codes': 5, 'include': 6, 'various': 7, 'covid': 8, 'may': 9, 'people': 10, 'develop': 11, 'players': 12, 'rules': 13, 'virus': 14, 'religious': 15, 'team': 16, 'sports': 17, 'kicking': 18, 'goal': 19, 'word': 20, 'rugby': 21, 'common': 22, 'games': 23, 'disease': 24, 'religions': 25, 'popular': 26, 'known': 27, 'league': 28, 'gaelic': 29, 'many': 30, 'public': 31, 'th': 32, 'century': 33, 'spread': 34, 'elements': 35, 'two': 36, 'also': 37, 'either': 38, 'foot': 39, 'severe': 40, 'risk': 41, 'contaminated': 42, 'transcription': 43, 'social': 44, 'beliefs': 45, 'supernatural': 46, 'sacred': 47, 'life': 48, 'varying': 49, 'form': 50, 'association': 51, 'american': 52, 'canadian': 53, 'australian': 54, 'union': 55, 'share': 56, 'origins': 57, 'traditional': 58, 'played': 59, 'different': 60, 'world': 61, 'cultural': 62, 'influence': 63, 'british': 64, 'empire': 65, 'end': 66, 'distinct': 67, 'developing': 68, 'first': 69, 'several

In [ ]:
# Convert them to tsv files in proper format
# as vector.tsv and as meta,tsv